In [89]:
from datetime import datetime
import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from PIL import Image
import html
from markdownify import markdownify 

#endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"
endpoint_url = "https://query.wikidata.org/sparql"

query_txt = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT ?textItem ?kuratorLabel ?textUrl
WHERE
{
  <placeholder>
  ?textItem cpsp:P46 ?kuratorStatement. 
  ?kuratorStatement cpsps:P46 ?kuratorItem. 
  ?kuratorItem rdfs:label ?kuratorLabel.
  ?textItem cpsp:P57 ?urlstatement. 
  ?urlstatement cpsps:P57 ?textUrl. 
}"""

query_img ="""
SELECT DISTINCT ?artwork ?artworkLabel ?image ?width ?height WHERE {
  ?artwork wdt:P31 wd:Q3305213;
    wdt:P170 wd:Q41406;
    wdt:P18 ?image;
    wdt:P2048 ?height;
    wdt:P2049 ?width.
  FILTER(?width > ?height)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY (?artworkLabel)
LIMIT 50
"""

query_graph = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT ?x ?y
WHERE
{
  ?a cpsp:P2 ?c.
  ?c cpsps:P2 ?d.
  ?a rdfs:label ?x.
  ?d rdfs:label ?y.

}LIMIT 1"""

query_graph2 = """
SELECT ?a ?b ?c
WHERE
{
  ?a rdfs:label ?c
}LIMIT 100"""


def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

"""def get_text(textitem_id):
    q = ""
    if textitem_id:
        q = query_txt.replace("<placeholder>", "cps:"+textitem_id+" cpsp:P46 ?kuratorStatement.")
    else:
        q = query_txt.replace("<placeholder>","")
    results_txt = run_query(endpoint_url, q)
    for item in results_txt["results"]["bindings"]:
        print('Wikibase link: ' + '[' + item['textItem']['value'] + ']' + '(' + item['textItem']['value'] + ')' + '\n')
        print('Kurator: ' + item['kuratorLabel']['value'] + '\n')
        headers = {'User-Agent': 'Ex_Books_conference_bot/0.0 (https://github.com/SimonXIX/Experimental_Books_workshop; ad7588@coventry.ac.uk)'}
        r = requests.get(item['textUrl']['value'], headers=headers, stream=True)
        text = str(r.text) #changed from r.content
        text = text.removeprefix("<!DOCTYPE html>") #changed from "b'<!DOCTYPE html>"
        print(markdownify(text)) #changed
"""
def get_delay(date):
    try:
        date = datetime.datetime.strptime(date, '%a, %d %b %Y %H:%M:%S GMT')
        timeout = int((date - datetime.datetime.now()).total_seconds())
    except ValueError:
        timeout = int(date)
    return timeout

# def fetch_image_by_url(url, headers):
#     r = requests.get(url, headers=headers, stream=True)
#     if r.status_code == 200:
#         im = Image.open(r.raw)
#         return im
#     if r.status_code == 500:
#         return None
#     if r.status_code == 403:
#         return None
#     if r.status_code == 429:
#         timeout = get_delay(r.headers['retry-after'])
#         print('Timeout {} m {} s'.format(timeout // 60, timeout % 60))
#         time.sleep(timeout)
#         fetch_image_by_url(url, headers)

def get_img(partOfItem_id):
    print("")
    print("<link rel='stylesheet' href='./section-fullimg.css'>")

    q = ""
    print("<div class='navig'>")

    print("<div class='arrow arrowleft'>&#10092;</div>")

    print("<div class='stop-scrolling'>")

    if partOfItem_id:
        q = query_img.replace("<placeholder>", "?partOfStatement cpsps:P6 cps:"+partOfItem_id+".")
    else:
        q = query_img.replace("<placeholder>","")
    results_img = run_query(endpoint_url, q)


    for item in results_img["results"]["bindings"]:    
      #print(item)
      #print('Wikibase link: ' + '[' + item['imgItem']['value'] + ']' + '(' + item['imgItem']['value'] + ')' + '\n')
      #print('Title: ' + item['itemLabel']['value'] + '\n')
      #print('Year: ' + item['publishDate']['value'] + '\n')
      #print('Description: ' + html.unescape(item['itemDescr']['value']) + '\n')

      # get image from image URL and display resized version


      image_url=item['image']['value']

      print(f'<img class="img_full" src="{image_url}"/>')

    print("</div>")
    onclicknext = f'openNext("{image_url}")'
    print(f"<div class='arrow arrowright' onclick='{onclicknext}'>&#10093;</div>")

    print("</div>")

    print('''
<script>
function openNext(imageUrl) {
  var image = document.getElementsByClassName("img_full");
  image.src = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Stray_kitten_Rambo002.jpg/1024px-Stray_kitten_Rambo002.jpg"
}
  
</script>
    ''')

    print(
    '''<div style='border:1px black solid; margin:1em;'>Learned to row a boat away with just one finger</br>
    Cast but prayers into the bow</br>
    I hope I sink her<br>
    Watch me fall asleep in a bag of rope<br>
    If the storm comes back, cut my throat<br>
    Calm the storm with all I must<br>
    I must a-whimper<br>
    <br>
    Cause I been holding out this hand<br>
    <i>I been holding out this hand</i><br>
    Old sock money bank<br>
    I spent all that I had<br>
    Now we go round and around and around<br>
    Oh, we go around and around and around<br>
    <br>
    I cut the storm with just a stick to watch it splinter<br>
    Now I'm begging ??? to make a home for winter<br>
    Drew my children in the snow <br>
    And spilled my blood to watch them grow<br>
    Cast me off this boat<br>
    It's me who sinks her<br>
    <br>
    Cause I been holding out this hand<br>
    <i>I been holding out this hand</i><br>
    Old sock money bank<br>
    I spent all that I had<br>
    Now we go round and around and around<br>
    Oh, we go around and around and around<br>
    Oh, we go around and around and around<br>
    Oh, we go around and around and around<br></div>''')



# """
# def get_graph():
#     import VizKG.visualize as vkg
#     results_graph1 = run_query(endpoint_url, query_graph)
#     #print(results_graph1)
#     #print('---')
#     results_graph2 = run_query(endpoint_url, query_graph2)
#     #print(results_graph2)

#     chart = vkg(sparql_query=query_graph2, sparql_service_url=endpoint_url, chart='wordcloud')
#     chart.plot()
# """

get_img("")



<link rel='stylesheet' href='./section-fullimg.css'>
<div class='navig'>
<div class='arrow arrowleft'>&#10092;</div>
<div class='stop-scrolling'>
<img class="img_full" src="http://commons.wikimedia.org/wiki/Special:FilePath/Edvard%20Munch%20-%2017th%20of%20May%20in%20a%20Small%2C%20Norwegian%20Town%20-%20MM.M.00681%20-%20Munch%20Museum.jpg"/>
<img class="img_full" src="http://commons.wikimedia.org/wiki/Special:FilePath/Edvard%20Munch%20-%2017th%20of%20May%20in%20a%20Small%2C%20Norwegian%20Town%20%281919%29.jpg"/>
<img class="img_full" src="http://commons.wikimedia.org/wiki/Special:FilePath/Edvard%20Munch%20-%20Aase%20and%20Harald%20N%C3%B8rregaard%20-%20NG.M.01794%20-%20National%20Museum%20of%20Art%2C%20Architecture%20and%20Design.jpg"/>
<img class="img_full" src="http://commons.wikimedia.org/wiki/Special:FilePath/Edvard%20Munch%20-%20Aasgaardstrand%20-%201974.57%20-%20Wadsworth%20Atheneum.jpg"/>
<img class="img_full" src="http://commons.wikimedia.org/wiki/Special:FilePath/Edvard%20Mu

In [90]:
#get_text("Q209")

In [91]:
#get_img("")

<hr>

In [92]:
#get_graph()